(lightning_mnist_example)=

# Train a Pytorch Lightning Image Classifier

This example introduces how to train a Pytorch Lightning Module using AIR {class}`LightningTrainer <ray.train.lightning.LightningTrainer>`. We will demonstrate how to train a basic neural network on the MNIST dataset with distributed data parallelism.


In [ ]:
!pip install "torchmetrics>=0.9" "pytorch_lightning>=1.6" 

In [ ]:
import os
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from filelock import FileLock
from torch.utils.data import DataLoader, random_split, Subset
from torchmetrics import Accuracy
from torchvision.datasets import MNIST
from torchvision import transforms

import pytorch_lightning as pl
from pytorch_lightning import trainer
from pytorch_lightning.core import datamodule
from pytorch_lightning.loggers.csv_logs import CSVLogger

## Prepare Dataset and Module

The Pytorch Lightning Trainer takes either `torch.utils.data.DataLoader` or `pl.LightningDataModule` as data inputs. You can keep using them without any changes for the Ray AIR LightningTrainer. 

In [2]:
class MNISTDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=100):
        super().__init__()
        self.data_dir = os.getcwd()
        self.batch_size = batch_size
        self.transform = transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
        )

    def setup(self, stage=None):
        with FileLock(f"{self.data_dir}.lock"):
            mnist = MNIST(
                self.data_dir, train=True, download=True, transform=self.transform
            )

            # split data into train and val sets
            self.mnist_train, self.mnist_val = random_split(mnist, [55000, 5000])

    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=self.batch_size, num_workers=4)

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=self.batch_size, num_workers=4)

    def test_dataloader(self):
        with FileLock(f"{self.data_dir}.lock"):
            self.mnist_test = MNIST(
                self.data_dir, train=False, download=True, transform=self.transform
            )
        return DataLoader(self.mnist_test, batch_size=self.batch_size, num_workers=4)


datamodule = MNISTDataModule(batch_size=128)

Next, define a simple multi-layer perception as the subclass of `pl.LightningModule`.

In [3]:
class MNISTClassifier(pl.LightningModule):
    def __init__(self, lr=1e-3, feature_dim=128):
        torch.manual_seed(421)
        super(MNISTClassifier, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, feature_dim),
            nn.ReLU(),
            nn.Linear(feature_dim, 10),
            nn.ReLU(),
        )
        self.lr = lr
        self.accuracy = Accuracy(task="multiclass", num_classes=10)
        self.eval_loss = []
        self.eval_accuracy = []

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.linear_relu_stack(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = torch.nn.functional.cross_entropy(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, val_batch, batch_idx):
        loss, acc = self._shared_eval(val_batch)
        self.log("val_accuracy", acc)
        self.eval_loss.append(loss)
        self.eval_accuracy.append(acc)
        return {"val_loss": loss, "val_accuracy": acc}

    def test_step(self, test_batch, batch_idx):
        loss, acc = self._shared_eval(test_batch)
        self.log("test_accuracy", acc)
        return {"test_loss": loss, "test_accuracy": acc}

    def _shared_eval(self, batch):
        x, y = batch
        logits = self.forward(x)
        loss = F.nll_loss(logits, y)
        acc = self.accuracy(logits, y)
        return loss, acc

    def on_validation_epoch_end(self):
        avg_loss = torch.stack(self.eval_loss).mean()
        avg_acc = torch.stack(self.eval_accuracy).mean()
        self.log("val_loss", avg_loss, sync_dist=True)
        self.log("val_accuracy", avg_acc, sync_dist=True)
        self.eval_loss.clear()
        self.eval_accuracy.clear()

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

You don't need to make any change to the definition of PyTorch Lightning model and datamodule.

(lightning-config-builder-intro)=

## Define the Cofigurations for AIR LightningTrainer

The {meth}`LightningConfigBuilder <ray.train.lightning.LightningConfigBuilder>` class stores all the parameters involved in training a PyTorch Lightning module. It takes the same parameter lists as those in PyTorch Lightning.

- The `.module()` method takes a subclass of `pl.LightningModule` and its initialization parameters. `LightningTrainer` will instantiate a model instance internally in the workers' training loop.
- The `.trainer()` method takes the initialization parameters of `pl.Trainer`. You can specify training configurations, loggers, and callbacks here.
- The `.fit_params()` method stores all the parameters that will be passed into `pl.Trainer.fit()`, including train/val dataloaders, datamodules, and checkpoint paths.
- The `.checkpointing()` method saves the configurations for a `RayModelCheckpoint` callback. This callback reports the latest metrics to the AIR session along with a newly saved checkpoint.
- The `.build()` method generates a dictionary that contains all the configurations in the builder. This dictionary will be passed to `LightningTrainer` later.

Next, let's go step-by-step to see how to convert your existing PyTorch Lightning training script to a LightningTrainer.

In [4]:
from pytorch_lightning.callbacks import ModelCheckpoint
from ray.air.config import RunConfig, ScalingConfig, CheckpointConfig
from ray.train.lightning import (
    LightningTrainer,
    LightningConfigBuilder,
    LightningCheckpoint,
)


def build_lightning_config_from_existing_code(use_gpu):
    # Create a config builder to encapsulate all required parameters.
    # Note that model instantiation and fitting will occur later in the LightingTrainer,
    # rather than in the config builder.
    config_builder = LightningConfigBuilder()

    # 1. define your model
    # model = MNISTClassifier(lr=1e-3, feature_dim=128)
    config_builder.module(cls=MNISTClassifier, lr=1e-3, feature_dim=128)

    # 2. define a ModelCheckpoint callback
    # checkpoint_callback = ModelCheckpoint(
    #     monitor="val_accuracy", mode="max", save_top_k=3
    # )
    config_builder.checkpointing(monitor="val_accuracy", mode="max", save_top_k=3)

    # 3. Define a Lightning trainer
    # trainer = pl.Trainer(
    #     max_epochs=10,
    #     accelerator="cpu",
    #     strategy="ddp",
    #     log_every_n_steps=100,
    #     logger=CSVLogger("logs"),
    #     callbacks=[checkpoint_callback],
    # )
    config_builder.trainer(
        max_epochs=10,
        accelerator="gpu" if use_gpu else "cpu",
        log_every_n_steps=100,
        logger=CSVLogger("logs"),
    )
    # You do not need to provide the checkpoint callback and strategy here,
    # since LightningTrainer configures them automatically.
    # You can also add any other callbacks into LightningConfigBuilder.trainer().

    # 4. Parameters for model fitting
    # trainer.fit(model, datamodule=datamodule)
    config_builder.fit_params(datamodule=datamodule)

    # Finally, compile all the configs into a dictionary for LightningTrainer
    lightning_config = config_builder.build()
    return lightning_config

Now put everything together:

In [ ]:
# Set it to False if you want to run without GPUs
use_gpu = True

In [6]:
lightning_config = build_lightning_config_from_existing_code(use_gpu=use_gpu)

scaling_config = ScalingConfig(num_workers=4, use_gpu=use_gpu)

run_config = RunConfig(
    name="ptl-mnist-example",
    storage_path="/tmp/ray_results",
    checkpoint_config=CheckpointConfig(
        num_to_keep=3,
        checkpoint_score_attribute="val_accuracy",
        checkpoint_score_order="max",
    ),
)

trainer = LightningTrainer(
    lightning_config=lightning_config,
    scaling_config=scaling_config,
    run_config=run_config,
)

Now fit your trainer:

In [8]:
result = trainer.fit()
print("Validation Accuracy: ", result.metrics["val_accuracy"])
result

find: ‘.git’: No such file or directory
2023-04-28 09:30:43,657	INFO worker.py:1432 -- Connecting to existing Ray cluster at address: 10.0.12.241:6379...
2023-04-28 09:30:43,665	INFO worker.py:1607 -- Connected to Ray cluster. View the dashboard at https://console.anyscale-staging.com/api/v2/sessions/ses_vhpce9uvpnmhikmask3c5db399/services?redirect_to=dashboard 
2023-04-28 09:30:43,671	INFO packaging.py:347 -- Pushing file package 'gcs://_ray_pkg_c896ee9346ecab5d19a2dbcff95e2084.zip' (0.07MiB) to Ray cluster...
2023-04-28 09:30:43,672	INFO packaging.py:360 -- Successfully pushed file package 'gcs://_ray_pkg_c896ee9346ecab5d19a2dbcff95e2084.zip'.
2023-04-28 09:30:43,725	INFO tune.py:221 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


(pid=56808) /mnt/cluster_storage/pypi/lib/python3.9/site-packages/neptune/common/warnings.py:62: NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
(pid=56808)   warnings.warn(
(LightningTrainer pid=56808) 2023-04-28 09:31:00,123	INFO backend_executor.py:128 -- Starting distributed worker processes: ['57429 (10.0.12.241)', '57430 (10.0.12.241)', '57431 (10.0.12.241)', '57432 (10.0.12.241)']
(RayTrainWorker pid=57429) 2023-04-28 09:31:01,088	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=4]
(RayTrainWorker pid=57431)   warnings.warn(
(RayTrainWorker pid=57431)   warnings.warn(
(RayTrainWorker pid=57429) /mnt/cluster_storage/pypi/lib/python3.9/site-packages/neptune/common/warnings.py:62: NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be remo

(RayTrainWorker pid=57432) Downloading http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz


  0%|          | 0/9912422 [00:00<?, ?it/s]


(RayTrainWorker pid=57432) Downloading http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz to /home/ray/default/doc/source/train/examples/lightning/MNIST/raw/train-images-idx3-ubyte.gz


...

(RayTrainWorker pid=57432) Missing logger folder: logs/lightning_logs
(RayTrainWorker pid=57431) LOCAL_RANK: 2 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
(RayTrainWorker pid=57429)   | Name              | Type       | Params
(RayTrainWorker pid=57429) -------------------------------------------------
(RayTrainWorker pid=57429) 0 | linear_relu_stack | Sequential | 101 K 
(RayTrainWorker pid=57429) 1 | accuracy          | Accuracy   | 0     
(RayTrainWorker pid=57429) -------------------------------------------------
(RayTrainWorker pid=57429) 101 K     Trainable params
(RayTrainWorker pid=57429) 0         Non-trainable params
(RayTrainWorker pid=57429) 101 K     Total params
(RayTrainWorker pid=57429) 0.407     Total estimated model params size (MB)
(RayTrainWorker pid=57431) [W reducer.cpp:1298] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag results in an extra traversal of the autograd graph every ite

Trial name,_report_on,date,done,epoch,experiment_tag,hostname,iterations_since_restore,node_ip,pid,should_checkpoint,step,time_since_restore,time_this_iter_s,time_total_s,timestamp,train_loss,training_iteration,trial_id,val_accuracy,val_loss
LightningTrainer_0593e_00000,train_epoch_end,2023-04-28_09-31-29,True,9,0,ip-10-0-12-241,10,10.0.12.241,56808,True,1080,33.056,1.58153,33.056,1682699489,0.0840481,10,0593e_00000,0.970436,-12.5445


2023-04-28 09:31:32,674	INFO tune.py:1010 -- Total run time: 48.95 seconds (48.90 seconds for the tuning loop).


Validation Accuracy:  0.9704360961914062


Result(
  metrics={'_report_on': 'train_epoch_end', 'train_loss': 0.0840480849146843, 'val_accuracy': 0.9704360961914062, 'val_loss': -12.544519424438477, 'epoch': 9, 'step': 1080, 'should_checkpoint': True, 'done': True, 'trial_id': '0593e_00000', 'experiment_tag': '0'},
  path='/tmp/ray_results/ptl-mnist-example/LightningTrainer_0593e_00000_0_2023-04-28_09-30-46',
  checkpoint=LightningCheckpoint(local_path=/tmp/ray_results/ptl-mnist-example/LightningTrainer_0593e_00000_0_2023-04-28_09-30-46/checkpoint_000009)
)

## Test your network on the test data

Next, we use PyTorch Lightning's native interface to evaluate the best model: To run the test loop using the ``pl.LightningModule.test_step()`` in your user-defined code, simply pass the loaded model to ``pl.Trainer.test()``. 

In [9]:
checkpoint: LightningCheckpoint = result.checkpoint
best_model: pl.LightningModule = checkpoint.get_model(MNISTClassifier)

In [10]:
trainer = pl.Trainer()
test_dataloader = datamodule.test_dataloader()
result = trainer.test(best_model, dataloaders=test_dataloader)

/home/ray/anaconda3/lib/python3.9/site-packages/pytorch_lightning/loops/utilities.py:92: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/ray/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1814: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=4)`.
  rank_zero_warn(
Missing logger folder: /home/ray/default/doc/source/train/examples/lightning/lightning_logs


Testing: 0it [00:00, ?it/s]

2023-04-28 09:31:33.611773: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 09:31:33.762802: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-28 09:31:34.628099: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-04-28 09:31:34.628189: W tensorflow/

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.9735999703407288     │
└───────────────────────────┴───────────────────────────┘

You can also use {class}`LightningPredictor <ray.train.lightning.LightningPredictor>` for inference.

In [11]:
from ray.train.lightning import LightningPredictor

predictor = LightningPredictor.from_checkpoint(
    checkpoint, MNISTClassifier, use_gpu=use_gpu
)


def accuracy(logits, labels):
    preds = np.argmax(logits, axis=1)
    correct_preds = np.sum(preds == labels)
    return correct_preds


corrects = total = 0
for batch in test_dataloader:
    inputs, labels = batch
    inputs, labels = inputs.numpy(), labels.numpy()
    logits = predictor.predict(inputs)["predictions"]
    total += labels.size
    corrects += accuracy(logits, labels)

print("Accuracy: ", corrects / total)

Accuracy:  0.9736


## What's next?

- {ref}`Use LightningTrainer with Ray Data and Batch Predictor <lightning_advanced_example>`
- {ref}`Hyperparameter searching with LightningTrainer + Ray Tune. <tune-pytorch-lightning-ref>`